<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/Bee_Database_Write.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2020-04-08 Erik Samsoe

* Link to source
* Identify file type
* Convert file
* Remove commas
* Load into BigQuery

In [0]:
import pandas as pd

* "MPG_textport.txt" shared by H through "Box".  
* Marirose shared the link in Slack with Erik.
* Original File is hosted on Drive here: [MPG_textport.txt](https://drive.google.com/open?id=1-4VDXNMHJpgoB_BZENVhF3BaGJyYBBoI)
* This file is loaded into a text editor.  VIM in this case.  The .txt file appears to be in tsv format.
  * Remove commas with search and replace
  * Replace tabs with commas
* Upload file to Notebook (because of large size)
* Check file type

In [0]:
# MPG_textport_nocommas.csv
src = 'MPG_textport_nocommas.csv'

In [0]:
df = pd.read_csv(src, encoding='utf-16')

In [0]:
df.head(2)

,BarcodeID,OtherCATALOG,Questionable_recordDATA,NestingDATA_Avalible,MicrosatDATA_Avalible,#ofmeasurements,#ofPHOTOS,Floralspec_INDEX,#of_Determinations,Specimen_status,modifier,typeSTATUS,Gynandromorph,Stylops,Mites,M,F,Queen,Nomen_key,GenusName,Subgenus,Species,Subspecies,AuthorDATE,MappedGenus,Mapped_subgenus,MappedSpeceis,MappedBEE,determiner,JD_date,date,StartDay,startMon,Year0,endDay,endMon,Year1,starttime,endtime,DecimalLat,DecimalLon,Location Description,Location_Name,GAP_mapped,ECO_mapped,ECOEPA_name,GRID_link_B,Level0_name,Level1_name,Level2_Name,Elev_meters,Radius of collection in meters,Eventcode,CollMeth,CollMeth_modifier,collectors,ProjectName,ProjectDATA_Flag,ProjectID,Floral_association,bbsl_oldcode,code,Conditions,barmetricstart,barmetricend,windspeedstart,windspeedend,tempstart,tempend,noteblock,datecreated
0,BBSL1000000,NaN,0,no,no,NaN,NaN,1.0,NaN,Pinned specimen,none,None,False,False,False,0,1,False,valid,Lasioglossum,(Evylaeus),cooleyi,NaN,(Crawford 1906),Lasioglossum,(Evylaeus),cooleyi,1.0,S.J. Burrows 2016,124,4 May 2015,4.0,May,2015.0,NaN,NaN,NaN,8:25:00 AM,3:51:00 PM,46.6812,-114.03677,MPG Ranch: Plot 240,MPG240,NaN,Montana Valley and Foothill grasslands ...,Middle Rockies ...,1533229.0,USA,Montana,Missoula,983.0,NaN,MPG-032,pt,Zero out of 21,E. Israel,MPG RANCH,NaN,101,No Floral Record,Logan,BBSL,Partly Cloudy (>50% sun),NaN,NaN,NaN,NaN,NaN,NaN,Notes-::Question notes-; mapped juristictions>...,2016-01-10 22:49:43.843000000
1,BBSL1000001,NaN,0,no,no,NaN,NaN,1.0,NaN,Pinned specimen,none,None,False,False,False,0,1,False,valid,Lasioglossum,(Evylaeus),cooleyi,NaN,(Crawford 1906),Lasioglossum,(Evylaeus),cooleyi,1.0,S.J. Burrows 2016,124,4 May 2015,4.0,May,2015.0,NaN,NaN,NaN,8:25:00 AM,3:51:00 PM,46.6812,-114.03677,MPG Ranch: Plot 240,MPG240,NaN,Montana Valley and Foothill grasslands ...,Middle Rockies ...,1533229.0,USA,Montana,Missoula,983.0,NaN,MPG-032,pt,Zero out of 21,E. Israel,MPG RANCH,NaN,101,No Floral Record,Logan,BBSL,Partly Cloudy (>50% sun),NaN,NaN,NaN,NaN,NaN,NaN,Notes-::Question notes-; mapped juristictions>...,2016-01-10 22:49:43.850000000


In [0]:
df.astype({'Year0': 'int'})

#df.Year0.unique()

ValueError: ignored

In [0]:
df.Year0.unique()

array([2015., 2014., 2016., 2017., 2018.,   nan, 2013.])

# Load Data

In [0]:
src = 'MPG_bees_20-04-08.csv'

In [0]:
df = pd.read_csv(src)

### Rename columns 
* to comply with BigQuery naming conventions

In [0]:
df.columns = ['BarcodeID', 'OtherCATALOG', 'Questionable_recordDATA',
       'NestingDATA_Avalible', 'MicrosatDATA_Avalible', 'num_ofmeasurements',
       'num_ofPHOTOS', 'Floralspec_INDEX', 'num_of_Determinations',
       'Specimen_status', 'modifier', 'typeSTATUS', 'Gynandromorph', 'Stylops',
       'Mites', 'M', 'F', 'Queen', 'Nomen_key', 'GenusName', 'Subgenus',
       'Species', 'Subspecies', 'AuthorDATE', 'MappedGenus', 'Mapped_subgenus',
       'MappedSpeceis', 'MappedBEE', 'determiner', 'JD_date', 'date',
       'StartDay', 'startMon', 'Year0', 'endDay', 'endMon', 'Year1',
       'starttime', 'endtime', 'DecimalLat', 'DecimalLon',
       'Location_Description', 'Location_Name', 'GAP_mapped', 'ECO_mapped',
       'ECOEPA_name', 'GRID_link_B', 'Level0_name', 'Level1_name',
       'Level2_Name', 'Elev_meters', 'Radius_of_collection_in_meters',
       'Eventcode', 'CollMeth', 'CollMeth_modifier', 'collectors',
       'ProjectName', 'ProjectDATA_Flag', 'ProjectID', 'Floral_association',
       'bbsl_oldcode', 'code', 'Conditions', 'barmetricstart', 'barmetricend',
       'windspeedstart', 'windspeedend', 'tempstart', 'tempend', 'noteblock',
       'datecreated']

### Set datatypes

# BigQuery

### Authenticate

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-bigquery-storage
!pip install --upgrade google-cloud-core

In [0]:
#@title Enter GCP/BigQuery Project ID
project_id = 'mpg-bees' #@param{type:"string"}

# Packages used for interfacing w/ BigQuery from Python
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

# Create BigQuery storage client
bq_storage_client = bigquery_storage_v1beta1.BigQueryStorageClient()

### Write to BigQuery

In [0]:
#@title Write All Weather Station Data to BigQuery Table
#project = 'mpg-bees' #@param{type:'string'}
output_dataset_id = 'bee_biology_systematics_lab' #@param{type:'string'}

output_table_id = 'dev_apr' #@param{type:'string'}

replace_or_append_output = 'replace' #@param{type:'string'} ['replace', 'append']

# Combine project and dataset
project_dataset = (bq_client.project + '.' + output_dataset_id)

# Check to make sure output dataset exists, create it if not
try:
 bq_client.get_dataset(output_dataset_id)
 print("Dataset " + project_dataset + " exists\n")
 
except:
 print("Dataset " + project_dataset + " doesn't exist, so creating it\n")
 dataset = bq_client.create_dataset(bigquery.Dataset(project_dataset))

job_config = bigquery.LoadJobConfig()

# Modify job config depending on if we want to replace or append to table
if(replace_or_append_output == 'replace'):
 job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE  
else:  
 job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

dataset_ref = bq_client.dataset(output_dataset_id)
table_ref = dataset_ref.table(output_table_id)

# # Get timestamp (UTC), add to data frame at granularity of seconds
# all_station_fit_info['timestamp'] = pd.Timestamp.now(tz = 'UTC').ceil(freq = 's'
#  )

# Use client functionality to load BigQuery table from Pandas data frame
bq_client.load_table_from_dataframe(
 dataframe = df,
 destination = table_ref,
 job_config = job_config
 ).result()
 
# print('All Station Fit Info output (' + replace_or_append_output + ') to ' +
#  project_dataset + '.' + output_table_id +
#  '\n')

Dataset mpg-bees.bee_biology_systematics_lab exists



In [0]:
bq_client.project

'mpg-bees'